## Introducción al Machine Learning 
### Importancia de Caracteristicas 

Este cuaderno muestra un ejemplo de cómo calcular la importancia de las características de un clasificador entrenado

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.metrics import confusion_matrix

In [ ]:
bank = pd.read_csv('../data/bank.csv', sep=";")

In [ ]:
features = bank.drop(columns='y')
features

Hacemos las transformaciones para tener todas las características numéricas
- Un ordinal para education
- Variables binarias (0-1) para todas las nominales (yes/no)

In [ ]:
educ_mapping = {'unknown':0,
                'primary':1,
                'secondary':2, 
                'tertiary':3}
features.education.replace(educ_mapping, inplace=True)

Podemos hacer el replace sobre todo el dataframe, porque no hay otros valores que coincidan

In [ ]:
yes_no_map = {
    'yes': 1,
    'no': 0
}
features.replace(yes_no_map, inplace=True)
features.head()

In [ ]:
scale_fts = ['age','balance','day','duration','pdays']
features.loc[:, scale_fts] = StandardScaler().fit_transform(features.loc[:,scale_fts])
minmax_fts = ['education','campaign','previous']
features.loc[:, minmax_fts] = MinMaxScaler().fit_transform(features.loc[:, minmax_fts])
features.head()

eliminaremos la variable 'month' por un tema de rendimiento

In [ ]:
features.drop('month', axis=1, inplace=True)

In [ ]:
features = pd.get_dummies(features)
features.head()

In [ ]:
target = bank.y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(features,
                                                    target,
                                                    test_size=0.4,
                                                    stratify=target)

In [ ]:
dtree = DecisionTreeClassifier(max_depth=7)
_ = dtree.fit(x_train, y_train)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
_ = knn.fit(features.values, target)

In [ ]:
knn.predict(x_train)

El accuracy de base.  Tomaremos este valor como referencia 
respecto al cálculo de la importancia de características

In [ ]:
print(dtree.score(x_train, y_train))
print(knn.score(x_train, y_train))

___

In [ ]:
from sklearn.inspection import permutation_importance

Calculamos la importancia de características por permutación utilizando
la función disponible en sklearn

In [ ]:
pimp_knn = permutation_importance(knn, x_train, y_train, n_repeats=10)
pimp_tree = permutation_importance(dtree, x_train, y_train, n_repeats=10)

In [ ]:
imp_df = pd.DataFrame({'ft': features.columns,
                       'tree_mean': pimp_tree.importances_mean,
                       'tree_std': pimp_tree.importances_std,
                       'knn_mean': pimp_knn.importances_mean,
                       'knn_std': pimp_knn.importances_std
                      })
imp_df.sort_values('tree_mean', ascending=False, inplace=True)

In [ ]:
imp_df_better = imp_df.iloc[:10]

In [ ]:
imp_df_better

In [ ]:
bw=0.25
plt.bar(np.arange(imp_df_better.shape[0]),
                  imp_df_better.tree_mean,
                  yerr=imp_df_better.tree_std,
                  width=bw,
                  tick_label=imp_df_better.ft,
                  )
plt.bar(np.arange(imp_df_better.shape[0])+bw,
                  imp_df_better.knn_mean,
                  yerr=imp_df_better.knn_std,
                  width=bw,
                  tick_label=imp_df_better.ft)
plt.legend(['Tree','Knn'])
plt.xticks(rotation=90)

___

### Ejercicio Propuesto
Realizar la importancia de características para el KNN, pero ahora partiendo de las características originales, sólo haciendo la transformación necesaria para las variables dummies
- Comparar el resultado
- Justificar la diferencia que ocurre por la diferencia de normalización y escalado de algunas características